# Topic 2: Basic Graphs

Now that you understand the fundamentals, let's build more sophisticated graphs with multiple nodes and complex state management.

## Learning Objectives

- Create graphs with multiple nodes
- Manage state transformations
- Build linear workflows
- Pass data between nodes effectively

In [ ]:
# Setup
import os
import getpass
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, SystemMessage

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")

model = ChatAnthropic(model="claude-3-5-sonnet-20241022")
print("✓ Setup complete!")

## Example: Content Creation Pipeline

Let's build a multi-step content creation workflow that:
1. Generates a topic outline
2. Writes content based on the outline
3. Reviews and improves the content

In [ ]:
# Define our state with more fields
class ContentState(TypedDict):
    topic: str
    outline: str
    content: str
    review: str
    final_content: str

print("✓ ContentState defined")

## Creating Multiple Nodes

Each node performs a specific task in our pipeline:

In [ ]:
def generate_outline(state: ContentState) -> ContentState:
    """Generate an outline for the topic."""
    print("📝 Generating outline...")
    
    prompt = f"""Create a detailed outline for a blog post about: {state['topic']}
    
    Include:
    - 3-4 main sections
    - Key points for each section
    - Estimated word count for each section
    
    Keep it concise but comprehensive."""
    
    response = model.invoke([HumanMessage(content=prompt)])
    
    return {
        "topic": state["topic"],
        "outline": response.content,
        "content": "",
        "review": "",
        "final_content": ""
    }

def write_content(state: ContentState) -> ContentState:
    """Write content based on the outline."""
    print("✍️  Writing content...")
    
    prompt = f"""Based on this outline, write a blog post:
    
    OUTLINE:
    {state['outline']}
    
    Write engaging, informative content. Keep it around 300-400 words."""
    
    response = model.invoke([HumanMessage(content=prompt)])
    
    return {
        "content": response.content
    }

def review_content(state: ContentState) -> ContentState:
    """Review and suggest improvements."""
    print("🔍 Reviewing content...")
    
    prompt = f"""Review this content and provide specific suggestions for improvement:
    
    CONTENT:
    {state['content']}
    
    Focus on:
    - Clarity and flow
    - Engagement
    - Completeness
    
    Be specific and actionable."""
    
    response = model.invoke([HumanMessage(content=prompt)])
    
    return {
        "review": response.content
    }

def finalize_content(state: ContentState) -> ContentState:
    """Apply review suggestions to create final version."""
    print("✨ Finalizing content...")
    
    prompt = f"""Based on this review, improve the content:
    
    ORIGINAL CONTENT:
    {state['content']}
    
    REVIEW:
    {state['review']}
    
    Provide the improved version."""
    
    response = model.invoke([HumanMessage(content=prompt)])
    
    return {
        "final_content": response.content
    }

print("✓ All nodes created!")

## Building the Graph

Now let's connect our nodes in a linear workflow:

In [ ]:
# Create the graph
graph_builder = StateGraph(ContentState)

# Add all nodes
graph_builder.add_node("generate_outline", generate_outline)
graph_builder.add_node("write_content", write_content)
graph_builder.add_node("review_content", review_content)
graph_builder.add_node("finalize_content", finalize_content)

# Add edges to create the flow
graph_builder.add_edge(START, "generate_outline")
graph_builder.add_edge("generate_outline", "write_content")
graph_builder.add_edge("write_content", "review_content")
graph_builder.add_edge("review_content", "finalize_content")
graph_builder.add_edge("finalize_content", END)

# Compile
content_graph = graph_builder.compile()

print("✓ Content creation graph compiled!")

## Visualize the Graph

In [ ]:
from IPython.display import Image, display

try:
    display(Image(content_graph.get_graph().draw_mermaid_png()))
except Exception:
    print("Graph structure:")
    print("START -> generate_outline -> write_content -> review_content -> finalize_content -> END")

## Run the Pipeline

Let's test our content creation pipeline:

In [ ]:
# Run the graph
result = content_graph.invoke({
    "topic": "How to Get Started with LangGraph",
    "outline": "",
    "content": "",
    "review": "",
    "final_content": ""
})

print("\n" + "="*50)
print("PIPELINE COMPLETE!")
print("="*50)

In [ ]:
# Display the outline
print("\n📝 OUTLINE:")
print("-" * 50)
print(result["outline"])

In [ ]:
# Display the first draft
print("\n✍️  FIRST DRAFT:")
print("-" * 50)
print(result["content"])

In [ ]:
# Display the review
print("\n🔍 REVIEW:")
print("-" * 50)
print(result["review"])

In [ ]:
# Display the final content
print("\n✨ FINAL CONTENT:")
print("-" * 50)
print(result["final_content"])

## Understanding State Updates

Notice how each node receives the full state and returns partial updates. LangGraph merges these updates automatically.

## Exercise: Build Your Own Pipeline

Create a research pipeline that:
1. Takes a research question
2. Generates key research areas to explore
3. Creates a research plan
4. Summarizes potential findings

Use the pattern above as a template!

In [ ]:
# Your code here!

class ResearchState(TypedDict):
    question: str
    research_areas: str
    research_plan: str
    summary: str

# TODO: Define your nodes
# TODO: Build your graph
# TODO: Run it with a research question


## Key Takeaways

In this notebook, you learned:

1. ✅ How to create graphs with multiple nodes
2. ✅ Managing complex state with multiple fields
3. ✅ Building linear workflows where data flows through stages
4. ✅ How nodes update state through partial returns
5. ✅ Creating practical multi-step pipelines

## Next Steps

Continue to **Topic 3: Conditional Edges** to learn how to add decision-making to your graphs!